In [1]:
import pandas as pd
# pd.options.display.max_colwidth = 200

import numpy as np
from collections import defaultdict

import matplotlib
import matplotlib.pyplot as plt

import seaborn as sns
sns.set_style("white")

from pymystem3 import Mystem; mystem = Mystem()
from functools import lru_cache


from tqdm import tqdm
tqdm.pandas()

%matplotlib inline

/home/sweetsteam/anaconda3/lib/python3.5/site-packages/matplotlib/__init__.py:878: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
data = []
DATA_PATH = '../../NewsParser/data/'
for csv_name in ['recent_news.csv']:
    data.append(pd.read_csv(DATA_PATH + csv_name))
data = pd.concat(data)

In [3]:
data.head()

,date,text,title,url
0,2017-04-18 19:41:14,"КИЕВ, 18 апреля. /ТАСС/. ""Дочка"" российского С...","""Дочка"" Сбербанка может оспорить запрет на исп...",http://tass.ru/ekonomika/4192151
1,2017-04-18 19:47:54,"ПАРИЖ, 18 апреля. /Корр. ТАСС Сергей Щербаков/...",Саркози призвал французских избирателей голосо...,http://tass.ru/mezhdunarodnaya-panorama/4192155
2,2017-04-18 19:37:26,"ЭЛЬ-КУВЕЙТ, 18 апреля. /Корр. ТАСС Павел Проко...","В Мосуле ликвидирован ""главный снайпер"" ИГ",http://tass.ru/mezhdunarodnaya-panorama/4192145
3,2017-04-18 19:59:38,"КИЕВ, 18 апреля. /ТАСС/. Национальный банк Укр...",Нацбанк Украины рассматривает документы от Nor...,http://tass.ru/ekonomika/4192182
4,2017-04-18 19:15:21,"РИМ, 18 апреля. /Корр.ТАСС Алексей Букалов/. Р...","Сильвио Берлускони потеснил ""Молодого папу"" в ...",http://tass.ru/kultura/4192052


In [4]:
class Pipeline(object):
    def __init__(self, *args):
        self.transformations = args
    def __call__(self, x):
        res = x
        for f in self.transformations:
            res = f(res)
        return res

from nltk.corpus import stopwords
from stop_words import get_stop_words
en_sw = get_stop_words('en')
ru_sw = get_stop_words('ru')
STOP_WORDS = set(en_sw) | set(ru_sw)
STOP_WORDS = STOP_WORDS | set(stopwords.words('russian')) | set(stopwords.words('english'))
STOP_WORDS = STOP_WORDS | set(['лента', 'новость', 'риа', 'тасс', 'редакция'])

def get_lower(text):
    return str(text).lower().strip()

def remove_punctuation(text):
    return ''.join([c if c.isalpha() or c in ['-',"'"] else ' ' for c in text])

@lru_cache(maxsize=None)
def get_word_normal_form(word):
    return ''.join(mystem.lemmatize(word)).strip().replace('ё', 'е').strip('-')

def lemmatize_words(text):
    res = []
    for word in text.split():
        norm_form = get_word_normal_form(word)
        if len(norm_form) > 2 and norm_form not in STOP_WORDS:
            res.append(norm_form)
    return ' '.join(res)

TEXT_PIPELINE = Pipeline(get_lower, remove_punctuation, lemmatize_words)

In [5]:
%%time
data.text = data.text.progress_apply(TEXT_PIPELINE)
data.title = data.title.apply(lambda x: x.strip())
data['title_norm'] = data.title.progress_apply(TEXT_PIPELINE)

100%|██████████| 927/927 [00:00<00:00, 11816.66it/s]

CPU times: user 1.65 s, sys: 176 ms, total: 1.82 s
Wall time: 3.78 s


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity 

In [7]:
trainX = data['title_norm'] + ' ' + data.text
# print(trainX)
trainX = trainX.values

In [14]:
trainX.shape

(927,)

In [10]:
import pickle

In [16]:
with open('../web_server/models/vectorizer.bin', 'rb') as pickle_file:
    tfidf_vectorizer = pickle.load(pickle_file)

In [61]:
%%time
# tfidf_vectorizer = TfidfVectorizer(min_df=5, ngram_range=(1,2), lowercase=False).fit(trainX)

CPU times: user 608 ms, sys: 4 ms, total: 612 ms
Wall time: 614 ms


In [17]:
len(tfidf_vectorizer.vocabulary_)

510889

In [18]:
tfidf_matrix = tfidf_vectorizer.transform(trainX)

# KMeans

In [19]:
from sklearn.cluster import KMeans
# from spherecluster import SphericalKMeans

In [20]:
kmeans = KMeans(n_clusters=30, random_state=42).fit(tfidf_matrix)
# kmeans = SphericalKMeans(n_clusters=K).fit(tfidf_matrix)

In [21]:
clasters = kmeans.predict(tfidf_matrix)
c_list = [ [] for i in range(30) ]
for i, claster in enumerate(clasters):
    tfidf_news = tfidf_matrix[i,:]
#     print(kmeans.cluster_centers_[claster].reshape(1, -1).shape, tfidf_news.shape)
    if cosine_similarity(tfidf_news, kmeans.cluster_centers_[claster].reshape(1, -1))[0][0] > 0.65:
        c_list[claster].append(i)

In [22]:
c_list

[[],
 [],
 [],
 [47, 212, 408, 618],
 [52, 65, 318, 646, 815],
 [15, 296, 324, 694, 856, 874],
 [],
 [],
 [37, 43, 70, 211, 487, 843, 901],
 [189],
 [],
 [],
 [],
 [],
 [],
 [766, 792],
 [],
 [],
 [],
 [309],
 [159, 441],
 [],
 [],
 [],
 [],
 [41, 410, 566, 625, 724, 809, 908],
 [746],
 [],
 [],
 []]

In [23]:
for i, group in enumerate(c_list):
    if len(group) < 3:
        continue
    print('Topic', i)
    for id_ in group:
        print(data.title[id_],data.url[id_])
    print()

Topic 3
Хлебопеки не нуждаются в законодательном запрете на возврат непроданного хлеба http://tass.ru/ekonomika/4191387
Роспотребнадзор поддержал запрет на возврат производителю непроданного хлеба http://tass.ru/ekonomika/4189937
Роспотребнадзор одобрил запрет на возврат непроданного хлеба https://lenta.ru/news/2017/04/18/bread/
Роспотребнадзор поддержал запрет на возврат просроченного хлеба производителю http://www.vedomosti.ru/business/news/2017/04/18/686198-rospotrebnadzor

Topic 4
Роспотребнадзор проверит отказ "Победы" пустить на рейс ребенка с ДЦП http://tass.ru/obschestvo/4191422
Росавиация назвала "овербукинг" причиной отказа "Победы" в перевозке ребенка с ДЦП http://tass.ru/obschestvo/4191799
"Победа" уволит сотрудника, не обеспечившего посадку на борт ребенка с ДЦП http://tass.ru/proisshestviya/4189442
Инцидентом с недопуском ребенка с ДЦП на рейс «Победы» занялась прокуратура http://www.vedomosti.ru/business/news/2017/04/18/686181-dtsp-pobedi
Прокуратура проверит отказ авиак

In [17]:
kmeans.cluster_centers_

array([[ 0.00021113,  0.        ,  0.00305723, ...,  0.00121998,
         0.00235024,  0.        ],
       [ 0.00171598,  0.0005434 ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.00743105],
       [ 0.00377684,  0.00134598,  0.        , ...,  0.        ,
         0.        ,  0.0012463 ],
       [ 0.        ,  0.0010983 ,  0.        , ...,  0.00031296,
         0.        ,  0.        ]])

# Graphs

In [24]:
cosines = []
for tfidf_news in tfidf_matrix:
    cosine = cosine_similarity(tfidf_news, tfidf_matrix)
    cosines.append(cosine.tolist()[0])

In [31]:
COS_THRESHOLD = 0.75
themes = [ -1 for _ in range(len(cosines)) ]
themes_ids = [ [] for _ in range(len(cosines)) ]
curr_theme = 0
for v, theme in enumerate(themes):
    if theme == -1:
        curr_theme += 1
        Q = []
        Q.append(v)
        themes[v] = curr_theme
        themes_ids[curr_theme].append(v)
        while Q:
            curr_v = Q.pop(0)
            for u, cos in enumerate(cosines[curr_v]):
                if cos >= COS_THRESHOLD and themes[u] == -1:
                    themes[u] = curr_theme
                    themes_ids[curr_theme].append(u)
                    Q.append(u)

In [32]:
# themes_ids

In [33]:
groups = sorted(themes_ids, key=lambda x: -len(x))

In [34]:
for i, group in enumerate(groups):
    if len(group) < 2:
        break
    print('Topic', i)
    for id_ in group:
        print(data.title[id_],data.url[id_])
    print()

Topic 0
Чистая прибыль банка «Россия» выросла на 75% за январь – март http://www.vedomosti.ru/finance/news/2017/04/18/686211-pribil-banka-rossiya
Квартальная прибыль «ВТБ 24» выросла в 3,9 раза http://www.vedomosti.ru/finance/news/2017/04/18/686204-pribil-vtb-24
Россельхозбанк снизил чистую прибыль в 7,6 раза за I квартал http://www.vedomosti.ru/finance/news/2017/04/18/686207-rosselhozbank-snizil-pribil

Topic 1
"Дочка" Сбербанка может оспорить запрет на использование своих торговых марок на Украине http://tass.ru/ekonomika/4192151
Киевский суд лишил Сбербанк доменного имени и запретил использовать торговую марку http://tass.ru/ekonomika/4192017

Topic 2
Хлебопеки не нуждаются в законодательном запрете на возврат непроданного хлеба http://tass.ru/ekonomika/4191387
"Пятерочка" уйдет от практики возврата непроданного хлеба производителям  c 1 июня http://tass.ru/ekonomika/4191118

Topic 3
Греф: бумажные трудовые книжки мешают работодателям и Пенсионному фонду http://tass.ru/ekonomika/419